<a href="https://colab.research.google.com/github/beimnet777/Breast-Cancer-Detection/blob/main/Breast_Disease_Detection_File_Preparation_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom

In [ ]:
import pydicom
import cv2
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
def change_mask_to_bw(image):
  img = image
  (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  dilated=cv2.dilate(im_bw,np.ones((3,3)),iterations=4)
  return dilated

In [ ]:
def find_contours(mask):
  contours, hierarchy = cv2.findContours(
      mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  x, y, w, h=None,None,None,None
  area=0
  for index,cnt in enumerate(contours):
    x_, y_, w_, h_ = cv2.boundingRect(cnt)
    new_area=abs(w_*h_)
    if new_area>area:
      area=new_area
      x, y, w, h=x_, y_, w_, h_
  return (w,h,x,y)

In [ ]:
def crop_image_(image,mask):
  # print(image.shape,mask.shape)
  
  bw_image=change_mask_to_bw(image)
  
  (w,h,x,y)=find_contours(bw_image)

  cropped_image = image[y:y+h, x:x+w]
  cropped_mask=mask[y:y+h, x:x+w]

  # print(cropped_image.shape)
  # cv2_imshow(cropped_image)
  # cv2_imshow(cropped_mask)
  return cropped_image,cropped_mask


In [ ]:
def mask_preparation(image,lower_red1, upper_red1,lower_red2, upper_red2):
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
  mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

  mask = mask1 + mask2

  image[mask > 0] = [255, 255, 255]

  image[mask == 0] = [0, 0, 0]

  # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  return image

In [ ]:
def get_medical_image(path):
  
  ds = pydicom.dcmread(path)

  pixel_data = ds.pixel_array

  pixel_data = (pixel_data - pixel_data.min()) / (pixel_data.max() - pixel_data.min())
  pixel_data = (pixel_data * 255).astype('uint8')


  return pixel_data

In [ ]:
def remove_texts(image):



  _, mask = cv2.threshold(image, 240, 255, cv2.THRESH_BINARY)


  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  mask = cv2.dilate(mask, kernel)


  image[mask == 255] = 0
  return image


In [ ]:
from glob import glob

In [ ]:
normal_path = '/content/drive/MyDrive/Dataset/Dataset/Normal_cases'
normal_paths = []
suspicious_path = '/content/drive/MyDrive/Dataset/Dataset/Suspicious_cases'
suspicious_paths = []

In [ ]:
dirs = next(os.walk(normal_path))[1]
for dir in dirs:
    paths = glob(os.path.join(normal_path,dir,"*.dcm"))
    normal_paths.extend(paths)

dirs = next(os.walk(suspicious_path))[1]
for dir in dirs:
    paths = glob(os.path.join(suspicious_path,dir,"*.dcm"))
    suspicious_paths.extend(paths)


In [ ]:
def prepare_dataset(paths,file_label,folder):
  
  for path in paths:
    print(folder,path)
    image = get_medical_image(path)
    mask_path = path.replace('.dcm','_GT.jpg')
    if os.path.exists(mask_path):
      mask = cv2.imread(mask_path)
      image=cv2.resize(image,(mask.shape[1],mask.shape[0]))
      image,mask=crop_image_(image,mask)

      lower_red1 = np.array([0, 120, 70])
      upper_red1 = np.array([10, 255, 255])

    
      lower_red2 = np.array([170, 120, 70])
      upper_red2 = np.array([180, 255, 255])

      cv2_imshow(mask)
      maks= mask_preparation(mask,lower_red1, upper_red1,lower_red2, upper_red2)
      cv2_imshow(mask)

      image_name = file_label+path.split('/')[-2]+"_"+path.split("/")[-1]
      image_name = image_name.replace('.dcm','.jpg')
      cv2.imwrite(os.path.join(folder,image_name),image)
      mask_name = image_name.replace('.jpg','_mask.jpg')
      print(folder,mask_name)
      cv2.imwrite(os.path.join(folder,mask_name),mask)

    

In [ ]:
prepare_dataset(normal_paths,"n_","/content/drive/MyDrive/Normal")

In [ ]:
prepare_dataset(suspicious_paths,"s_","/content/drive/MyDrive/Suspicious")